## Use Quepid via API

Objective:

Set up a working Quepid instance programmatically via unofficial HTTP API (develper in this repo).
As an example, we will use the WANDS dataset.

#### Config

In [37]:
WANDS_INDEX = 'http://localhost:9200/wands'
QUEPID_TOKEN = '56d958e1221fd7e683123f1d7317dbdd925618d9e193e958bdfc3fd319f04fae'

In [38]:
import requests
from tqdm import tqdm

## Set up Quepid

Folllow the instrutions in [README](https://github.com/frutik/quepid-api-unofficial?tab=readme-ov-file#run-locally-connecting-to-local-quepid)

## WANDS

In [1]:
!git clone https://github.com/wayfair/WANDS.git

Cloning into 'WANDS'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 40 (delta 6), reused 3 (delta 3), pack-reused 21 (from 1)
Receiving objects: 100% (40/40), 33.32 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [6]:
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import pandas as pd

In [40]:
query_df = pd.read_csv("WANDS/dataset/query.csv", sep='\t')
query_df

,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners
...,...,...,...
475,483,rustic twig,Faux Plants and Trees
476,484,nespresso vertuo next premium by breville with...,Espresso Machines
477,485,pedistole sink,Kitchen Sinks
478,486,54 in bench cushion,Furniture Cushions


In [4]:
product_df = pd.read_csv("WANDS/dataset/product.csv", sep='\t')
product_df

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0
...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,NaN,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0
42992,42992,fletcher 27.5 '' wide polyester armchair,Teen Lounge Furniture|Accent Chairs,Furniture / Living Room Furniture / Chairs & S...,"bring iconic , modern style to your space in a...",legmaterialdetails : rubberwood|backheight-sea...,1746.0,4.5,1226.0


### Create index

In [41]:
requests.put(
    WANDS_INDEX,
    json={
        "mappings": {
            "properties": {
                "name": {
                    "type": "text"
                },
                "description": {
                    "type": "text"
                }
            }
        }
    }
)


<Response [200]>

### Index data

In [42]:
def index_record(id, name, description):
    if id and name and description:
        try:
            return requests.post(
                f"{WANDS_INDEX}/_doc/{id}",
                json={
                    'name': name,
                    'description': description    
                }
            )
        except:
            pass

In [43]:
for index, row in tqdm(product_df.iterrows(), total=len(product_df)):
    _ = index_record(row['product_id'], row['product_name'], row['product_description'])

100%|███████████████████████████████████████████████████████████████| 42994/42994 [01:36<00:00, 446.24it/s]


In [44]:
response = requests.post(
    f"{WANDS_INDEX}/_search",
    json={
        "size": 0,
        "track_total_hits": True
    }
)
response.json()

{'took': 266,
 'timed_out': False,
 'terminated_early': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 36985, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

## Tests

### Create team

In [45]:
team = requests.post(
    'http://localhost:8081/api/teams/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "name": "wands"
    }   
)

team = team.json()

In [46]:
team

{'id': 1,
 'name': 'wands',
 'created_at': '2025-07-06T18:05:09.014Z',
 'updated_at': '2025-07-06T18:05:09.014Z'}

### Create search endpoint

In [47]:
endpoint = requests.post(
    'http://localhost:8081/api/search_endpoints/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "name": "wands",
        "endpoint_url": f"{WANDS_INDEX}/_search",
        "search_engine": "es",
        "api_method": "POST",
        "proxy_requests": 1,   
    }   
)

endpoint = endpoint.json()


In [48]:
endpoint

{'id': 1,
 'name': 'wands',
 'owner': 1,
 'search_engine': 'es',
 'endpoint_url': 'http://localhost:9200/wands/_search',
 'api_method': 'POST',
 'custom_headers': None,
 'archived': 0,
 'created_at': '2025-07-06T18:05:31.503Z',
 'updated_at': '2025-07-06T18:05:31.503Z',
 'basic_auth_credential': None,
 'mapper_code': None,
 'proxy_requests': 1,
 'options': None}

### Create case

In [51]:
case = requests.post(
    'http://localhost:8081/api/case/', 
    headers = {
        "Authorization": f"Bearer {QUEPID_TOKEN}"
    },
    json={
        "name": "wands",
        "scorer_id": 5,
        "book_id": 0,
        "search_endpoint_id": endpoint.get('id')
    }   
)

case = case.json()

In [52]:
case

{'id': 2,
 'case_name': 'wands',
 'last_try_number': 1,
 'owner': 1,
 'archived': 0,
 'scorer_id': 5,
 'created_at': '2025-07-06T18:06:31.262Z',
 'updated_at': '2025-07-06T18:06:31.262Z',
 'book_id': None,
 'public': None,
 'options': None,
 'nightly': None}

### Create queries

In [56]:
def add_query(case, query):
    return requests.post(
        f'http://localhost:8081/api/query/{case.get("id")}/', 
        headers = {
            "Authorization": f"Bearer {QUEPID_TOKEN}"
        },
        json={
            "query_text": query
        }   
    )

In [57]:
for index, row in tqdm(query_df.iterrows(), total=len(query_df)):
    add_query(case, row['query'])

100%|███████████████████████████████████████████████████████████████████| 480/480 [00:03<00:00, 142.44it/s]
